In [7]:
import numpy as np
import collections
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score
import matplotlib.pyplot as plt
import datetime


In [8]:
import os
os.listdir()

['.ipynb_checkpoints', 'bankTestX.csv', 'bankTrain.csv', 'Untitled.ipynb']

In [9]:
data= pd.read_csv('bankTrain.csv')

In [10]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,blue-collar,divorced,secondary,no,644,no,yes,cellular,17,nov,220,4,109,4,failure,no
1,52,admin.,married,primary,no,208,yes,no,cellular,17,apr,377,2,-1,0,unknown,no
2,34,technician,married,secondary,no,1641,yes,no,cellular,17,apr,380,1,-1,0,unknown,no
3,57,services,married,secondary,no,1433,no,no,telephone,21,nov,41,1,-1,0,unknown,no
4,57,technician,married,tertiary,no,2887,yes,no,cellular,11,feb,123,1,-1,0,unknown,no


In [11]:
from sklearn import preprocessing
for column_name in data.columns:
    if data[column_name].dtype == object:
        data[column_name] = data[column_name].astype('category')

In [12]:
data2 = pd.get_dummies(data, columns=['job', 'marital','education', 'default','housing','loan','contact','month','poutcome'])

In [13]:
data2.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,no,0,1,...,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,no,1,0,...,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,no,0,0,...,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,no,0,0,...,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,no,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
data.shape,data2.shape


((10000, 17), (10000, 52))

In [15]:
data2.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,no,0,1,...,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,no,1,0,...,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,no,0,0,...,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,no,0,0,...,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,no,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
data2.y.value_counts()

no     8854
yes    1146
Name: y, dtype: int64

In [17]:
train=data2[:7000]
test= data2[7000:]
train.shape, test.shape

((7000, 52), (3000, 52))

In [18]:
y_train= train.y 
X_train= train.drop(['y'], axis=1)
X_train.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
X_train.shape, y_train.shape

((7000, 51), (7000,))

In [20]:
y_test= test.y 
X_test= test.drop(['y'], axis=1)
X_test.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
7000,55,602,7,350,1,98,5,1,0,0,...,0,0,0,0,0,1,0,0,1,0
7001,51,837,30,41,2,-1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
7002,55,617,20,529,3,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7003,27,255,12,123,1,182,4,1,0,0,...,0,0,0,0,0,0,0,0,1,0
7004,29,2,15,65,3,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
X_test.shape, y_test.shape

((3000, 51), (3000,))

In [22]:
y_test.value_counts()

no     2649
yes     351
Name: y, dtype: int64

In [23]:
y_train.value_counts()

no     6205
yes     795
Name: y, dtype: int64

In [24]:
y_train.value_counts()

no     6205
yes     795
Name: y, dtype: int64

In [25]:
clf=  SGDClassifier()
clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [26]:
preds=clf.predict(X_train)
sum(preds==y_train) , len(y_train), sum(preds== y_train) / len(y_train)


(6228, 7000, 0.8897142857142857)

In [27]:
preds=clf.predict(X_test)
sum(preds==y_test) , len(y_test), sum(preds== y_test) / len(y_test)

(2634, 3000, 0.878)

In [28]:
preds

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype='<U3')

In [38]:


df = pd.DataFrame({'label':preds})

df.to_csv("cevirme.csv", encoding='utf-8', index=False)
counts=pd.read_csv("cevirme.csv")
counts.label.value_counts()

no     2850
yes     150
Name: label, dtype: int64

In [31]:
clf=  RandomForestClassifier()
clf.fit(X_train, y_train)

C:\Users\asus\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
preds=clf.predict(X_train)
sum(preds==y_train) , len(y_train), sum(preds== y_train) / len(y_train)


(6929, 7000, 0.9898571428571429)

In [34]:
preds=clf.predict(X_test)
sum(preds==y_test) , len(y_test), sum(preds== y_test) / len(y_test)

(2667, 3000, 0.889)

In [35]:
from pandas import ExcelWriter
from pandas import ExcelFile


df = pd.DataFrame({'label':preds})

writer = ExcelWriter('predsornek2.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()

counts=pd.read_excel("predsornek2.xlsx")
counts.label.value_counts()

no     2850
yes     150
Name: label, dtype: int64

In [36]:
preds

array(['yes', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

# SECOND PART

In [39]:
bankTestX= pd.read_csv("bankTestX.csv")

In [40]:
bankTestX.shape

(2001, 16)

In [41]:
bankTestX.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,39,unemployed,single,secondary,no,356,yes,no,unknown,30,may,462,2,-1,0,unknown
1,46,management,single,tertiary,yes,-344,no,no,cellular,8,aug,142,4,-1,0,unknown
2,35,management,single,tertiary,no,560,yes,no,unknown,6,may,181,1,-1,0,unknown
3,27,services,married,secondary,no,0,yes,no,unknown,15,may,312,6,-1,0,unknown
4,56,technician,married,secondary,no,2558,no,no,unknown,19,jun,288,1,-1,0,unknown


In [42]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,blue-collar,divorced,secondary,no,644,no,yes,cellular,17,nov,220,4,109,4,failure,no
1,52,admin.,married,primary,no,208,yes,no,cellular,17,apr,377,2,-1,0,unknown,no
2,34,technician,married,secondary,no,1641,yes,no,cellular,17,apr,380,1,-1,0,unknown,no
3,57,services,married,secondary,no,1433,no,no,telephone,21,nov,41,1,-1,0,unknown,no
4,57,technician,married,tertiary,no,2887,yes,no,cellular,11,feb,123,1,-1,0,unknown,no


In [43]:
data=pd.get_dummies(data, columns=['job', 'marital','education', 'default','housing','loan','contact','month','poutcome'])

In [44]:
data.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,no,0,1,...,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,no,1,0,...,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,no,0,0,...,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,no,0,0,...,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,no,0,0,...,0,0,0,0,0,0,0,0,0,1


In [45]:
bankTestX = pd.get_dummies(bankTestX,columns=['job', 'marital','education', 'default','housing','loan','contact','month','poutcome'] )

In [46]:
bankTestX.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,39,356,30,462,2,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,46,-344,8,142,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,35,560,6,181,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,27,0,15,312,6,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,56,2558,19,288,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [48]:
data.shape, bankTestX.shape

((10000, 52), (2001, 51))

In [49]:
train=data

In [50]:
y_train= train.y 
X_train= train.drop(['y'], axis=1)
X_train.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
test= bankTestX

In [52]:
test.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,39,356,30,462,2,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,46,-344,8,142,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,35,560,6,181,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,27,0,15,312,6,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,56,2558,19,288,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


# SGD Classifier

In [53]:
clf=  SGDClassifier()
clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [54]:
preds=clf.predict(X_train)
sum(preds==y_train) , len(y_train), sum(preds== y_train) / len(y_train)

(8342, 10000, 0.8342)

In [55]:
preds=clf.predict(test)

In [56]:
preds

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype='<U3')

In [57]:

df = pd.DataFrame({'label':preds})

df.to_csv("predssgdclassifier.csv", encoding='utf-8', index=False)
counts=pd.read_csv("predssgdclassifier.csv")
counts.label.value_counts()

no     1683
yes     318
Name: label, dtype: int64

# RandomForestClassifier

In [58]:
clf=  RandomForestClassifier()
clf.fit(X_train, y_train)

C:\Users\asus\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
preds=clf.predict(X_train)
sum(preds==y_train) , len(y_train), sum(preds== y_train) / len(y_train)

(9918, 10000, 0.9918)

In [60]:
preds=clf.predict(test)
preds

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [61]:

df = pd.DataFrame({'label':preds})

df.to_csv("predsrforest.csv", encoding='utf-8', index=False)
counts=pd.read_csv("predsrforest.csv")
counts.label.value_counts()

no     1886
yes     115
Name: label, dtype: int64

# Decision Tree

In [62]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [63]:
preds=clf.predict(X_train)
sum(preds==y_train) , len(y_train), sum(preds== y_train) / len(y_train)


(9062, 10000, 0.9062)

In [64]:
preds=clf.predict(test)
preds

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [65]:

df = pd.DataFrame({'label':preds})

df.to_csv("predsdtree.csv", encoding='utf-8', index=False)
counts=pd.read_csv("predsdtree.csv")
counts.label.value_counts()

no     1894
yes     107
Name: label, dtype: int64

In [74]:
print("abc")

abc


In [79]:
counts=pd.read_csv("fatihkran.csv")
counts.label.value_counts()

no     1878
yes     123
Name: label, dtype: int64